# Лабораторная 5: суммаризация с ruT5

Файнтюним `ai-forever/ruT5-base` на подзадачи генерации заголовков и кратких саммари по датасету Газеты.ру. Используем небольшую выборку (1000 обучающих и 200 тестовых примеров), ROUGE для оценки качества и простую схему с разными префиксами для двух задач.

In [2]:
import torch
from pathlib import Path
from __future__ import annotations

from dataclasses import dataclass
from typing import Optional

from datasets import DatasetDict, load_dataset
from transformers import (
    AutoTokenizer,
)
from lab5_utils import train_task, generate_sample

print("Torch:", torch.__version__)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

ModuleNotFoundError: No module named 'evaluate'

## Загрузка данных и базовой модели

In [ ]:
MODEL_NAME = "ai-forever/ruT5-base"
DATASET_SIZES = {"train": "train[:1000]", "test": "test[:200]"}
OUTPUT_ROOT = Path("models")
OUTPUT_ROOT.mkdir(exist_ok=True)

def load_gazeta_dataset(
    train_slice: str = "train[:1000]",
    test_slice: str = "test[:200]",
    cache_dir: Optional[Path] = None,
) -> DatasetDict:
    return load_dataset(
        "IlyaGusev/gazeta",
        revision="v2.0",
        split={"train": train_slice, "test": test_slice},
        cache_dir=str(cache_dir) if cache_dir else None,
    )

raw_dataset = load_gazeta_dataset(
    train_slice=DATASET_SIZES["train"],
    test_slice=DATASET_SIZES["test"],
)
print(raw_dataset)

DEFAULT_MODEL_NAME = "ai-forever/ruT5-base"
def load_tokenizer(model_name: str = DEFAULT_MODEL_NAME):
    return AutoTokenizer.from_pretrained(model_name)
tokenizer = load_tokenizer(MODEL_NAME)


In [ ]:
raw_dataset['train'][0]

## Настройка задач

Две отдельные задачи с разными целевыми полями и префиксами. Можно регулировать длину выходов и число эпох при необходимости.

In [ ]:
@dataclass
class TaskConfig:
    name: str
    target_field: str
    prefix: str
    max_target_length: int
    num_train_epochs: float = 3.0
    learning_rate: float = 3e-4
    batch_size: int = 4
    generation_max_length: int = 64
    num_beams: int = 4

tasks = [
    # Снимите комментарий, если нужно доучить summary заново
    # TaskConfig(
    #     name="summary",
    #     target_field="summary",
    #     prefix="summarize: ",
    #     max_target_length=96,
    #     generation_max_length=96,
    #     num_train_epochs=3,
    #     batch_size=4,
    # ),
    TaskConfig(
        name="title",
        target_field="title",
        prefix="title: ",
        max_target_length=32,
        generation_max_length=32,
        num_train_epochs=3,
        batch_size=4,
    ),
]


## Обучение и оценка

Запускаем обучение обеих моделей.

In [ ]:

metrics = {}
for task in tasks:
    print(f"Training {task.name}...")
    _, eval_metrics = train_task(
        raw_dataset=raw_dataset,
        tokenizer=tokenizer,
        task=task,
        output_root=OUTPUT_ROOT,
        model_name=MODEL_NAME,
    )
    metrics[task.name] = eval_metrics
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

metrics


## Генерация для произвольной новости

Укажите любой текст новости (можно взять пример из тестовой выборки) и получайте саммари и заголовок из обученных моделей.

In [ ]:
sample_text = raw_dataset["test"][10]["text"]

generated_summary = generate_sample(
    model_path=OUTPUT_ROOT / "summary",
    prefix="summarize: ",
    text=sample_text,
    max_length=96,
)

generated_title = generate_sample(
    model_path=OUTPUT_ROOT / "title",
    prefix="title: ",
    text=sample_text,
    max_length=32,
)

print("Source snippet:",sample_text[:500], "...")
print("Generated title:", generated_title)
print("Generated summary:", generated_summary)